In [2]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


import $exclude.$                        , $ivy.$                            // for cleaner logs

import $ivy.$                                   // adjust spark version - spark >= 2.0

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


In [3]:
import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._


import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._


In [4]:
val spark = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  .master("local[*]") // change to "yarn-client" on YARN
  .config("spark.executor.memory", "6g")
  .appName("NohupReader")
  .getOrCreate()

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


spark: SparkSession = org.apache.spark.sql.SparkSession@c971261

In [5]:
val filename = "/home/and/Documents/PhD/Research/Scripts/Python/test.out"
val nohup = spark.read.textFile(filename)

filename: String = "/home/and/Documents/PhD/Research/Scripts/Python/test.out"
nohup: Dataset[String] = [value: string]

In [6]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class Line(line: String, n: Long)
case class Param(runID: Long, date: String, method: String, cores: Int, epsilon: Double, mu: Int, delta: Int, methodTime: Double)
case class Stat(runID: Long, n: Long, timestamp: String, stage: String, stageTime: Double, load: Int, unit: String)

defined class Line
defined class Param
defined class Stat

In [7]:
import org.apache.spark.sql.functions._ 
import spark.implicits._

val lines = nohup.toDF("line").withColumn("n", monotonicallyIncreasingId).as[Line].cache()
val nLines = lines.count()

import org.apache.spark.sql.functions._ 

import spark.implicits._


lines: Dataset[Line] = [line: string, n: bigint]
nLines: Long = 2682L

In [8]:
val indices = lines.filter{ l => 
        l.line.contains("=== MergeLast Start ===") || l.line.contains("method=MergeLast,")
    }
    .orderBy("n")
    .select("n")
    .collect()
    .toList
    .map(_.getLong(0))
    .grouped(2)
    .toList
    .map(pair => (pair.head, pair.last))
    .filter(r => r._1 != r._2)
    .zipWithIndex
val index = spark.createDataset(indices)
    .flatMap{ pair => 
        (pair._1._1 to pair._1._2)
        .toList.map(v => (pair._2, v))
    }
    .toDF("runID","n")
    .cache

indices: List[((Long, Long), Int)] = List(((26L, 659L), 0), ((907L, 1540L), 1), ((1788L, 2421L), 2))
index: Dataset[Row] = [runID: int, n: bigint]

In [9]:
val runs = index.join(lines, "n").cache

runs: Dataset[Row] = [n: bigint, runID: int ... 1 more field]

In [10]:
val params = runs.groupBy("runID")
    .agg(max($"n").alias("n"))
    .join(lines, "n")
    .select("runID", "line")
    .orderBy("runID")
    .map{ row =>
        val runID = row.getInt(0)
        val line  = row.getString(1)
        var arr1  = line.split(" -> ")
        val date  = arr1(0)
        val arr2  = arr1(1).split(",")
        val method  = arr2(0).split("=")(1)
        val cores   = arr2(1).split("=")(1).toInt
        val epsilon = arr2(2).split("=")(1).toDouble
        val mu      = arr2(3).split("=")(1).toInt
        val delta   = arr2(4).split("=")(1).toInt
        val time    = arr2(5).split("=")(1).toDouble
        Param(runID, date, method, cores, epsilon, mu, delta, time)
    }
    .cache
params.show(10, truncate = false)

+-----+-----------------------+---------+-----+-------+---+-----+----------+
|runID|date                   |method   |cores|epsilon|mu |delta|methodTime|
+-----+-----------------------+---------+-----+-------+---+-----+----------+
|0    |2018-05-12 09:34:11,137|MergeLast|28   |10.0   |4  |5    |1431.151  |
|1    |2018-05-12 10:12:40,777|MergeLast|28   |20.0   |4  |5    |1851.144  |
|2    |2018-05-12 11:05:19,245|MergeLast|28   |30.0   |4  |5    |2484.781  |
+-----+-----------------------+---------+-----+-------+---+-----+----------+



params: Dataset[Param] = [runID: bigint, date: string ... 6 more fields]

In [11]:
val stats = runs.filter(_.getString(2).contains("|"))
    .map{ row =>
        val n     = row.getLong(0)
        val runID = row.getInt(1)
        val line  = row.getString(2)
        var arr1  = line.split(" -> ")
        val timestamp  = arr1(0).trim
        val arr2  = arr1(1).split("\\|")
        val stage = arr2(0).trim
        val time  = arr2(1).trim.dropRight(1).toDouble
        val arr3  = arr2(2).trim.split(" ")
        val load  = arr3(0).toInt
        val unit  = arr3(1)
        Stat(runID, n, timestamp, stage, time, load, unit)
    }.cache
stats.count()
stats.show(10, truncate=false)

+-----+---+-----------------------+------------------------------+---------+----+------------+
|runID|n  |timestamp              |stage                         |stageTime|load|unit        |
+-----+---+-----------------------+------------------------------+---------+----+------------+
|0    |57 |2018-05-12 09:13:01,801|1.Getting disks               |161.63   |30  |disks       |
|0    |58 |2018-05-12 09:13:05,945|2.Joining timestams           |4.14     |10  |candidates  |
|0    |77 |2018-05-12 09:13:22,486|3.Distance Join phase...      |3.87     |13  |combinations|
|0    |78 |2018-05-12 09:13:24,676|4.Getting candidates...       |2.19     |10  |candidates  |
|0    |98 |2018-05-12 09:13:42,017|3.Distance Join phase...      |4.06     |15  |combinations|
|0    |99 |2018-05-12 09:13:44,276|4.Getting candidates...       |2.26     |10  |candidates  |
|0    |119|2018-05-12 09:14:00,637|3.Distance Join phase...      |4.28     |18  |combinations|
|0    |120|2018-05-12 09:14:03,010|4.Getting candi

stats: Dataset[Stat] = [runID: bigint, n: bigint ... 5 more fields]
res10_1: Long = 192L

In [14]:
val data = params.join(stats, "runID").orderBy("n").cache
data.count()
data.show(10)

+-----+--------------------+---------+-----+-------+---+-----+----------+---+--------------------+--------------------+---------+----+------------+
|runID|                date|   method|cores|epsilon| mu|delta|methodTime|  n|           timestamp|               stage|stageTime|load|        unit|
+-----+--------------------+---------+-----+-------+---+-----+----------+---+--------------------+--------------------+---------+----+------------+
|    0|2018-05-12 09:34:...|MergeLast|   28|   10.0|  4|    5|  1431.151| 57|2018-05-12 09:13:...|     1.Getting disks|   161.63|  30|       disks|
|    0|2018-05-12 09:34:...|MergeLast|   28|   10.0|  4|    5|  1431.151| 58|2018-05-12 09:13:...| 2.Joining timestams|     4.14|  10|  candidates|
|    0|2018-05-12 09:34:...|MergeLast|   28|   10.0|  4|    5|  1431.151| 77|2018-05-12 09:13:...|3.Distance Join p...|     3.87|  13|combinations|
|    0|2018-05-12 09:34:...|MergeLast|   28|   10.0|  4|    5|  1431.151| 78|2018-05-12 09:13:...|4.Getting cand

data: Dataset[Row] = [runID: bigint, date: string ... 12 more fields]
res13_1: Long = 192L

In [16]:
val d = data.collect.sortBy(_.getLong(8)).map(_.mkString(";")).mkString("\n")
val research_home: String = scala.util.Properties.envOrElse("RESEARCH_HOME", "/home/and/Documents/PhD/Research/")
val path = "Experiments/Logs/"

import java.io._
val pw = new PrintWriter(new File(s"${research_home}${path}output.csv" ))
pw.write(s"$d\n")
pw.close

d: String = """
0;2018-05-12 09:34:11,137;MergeLast;28;10.0;4;5;1431.151;57;2018-05-12 09:13:01,801;1.Getting disks;161.63;30;disks
0;2018-05-12 09:34:11,137;MergeLast;28;10.0;4;5;1431.151;58;2018-05-12 09:13:05,945;2.Joining timestams;4.14;10;candidates
0;2018-05-12 09:34:11,137;MergeLast;28;10.0;4;5;1431.151;77;2018-05-12 09:13:22,486;3.Distance Join phase...;3.87;13;combinations
0;2018-05-12 09:34:11,137;MergeLast;28;10.0;4;5;1431.151;78;2018-05-12 09:13:24,676;4.Getting candidates...;2.19;10;candidates
0;2018-05-12 09:34:11,137;MergeLast;28;10.0;4;5;1431.151;98;2018-05-12 09:13:42,017;3.Distance Join phase...;4.06;15;combinations
0;2018-05-12 09:34:11,137;MergeLast;28;10.0;4;5;1431.151;99;2018-05-12 09:13:44,...
research_home: String = "/home/and/Documents/PhD/Research/"
path: String = "Experiments/Logs/"
import java.io._

pw: PrintWriter = java.io.PrintWriter@5d68fd21